### ResNetの実装
ResNetの基本式 $y = F(x) + x$
ここで $F(x)$ は残差関数

$$
\text{Output} = \text{ReLU}(F(x) + x)
$$

### BatchNormalization()の中身について
> ReLUにそのまま通してしまったら負数の情報が全て0になってしまうので、標準化を行う
> $$ \hat{x} = \frac{x - \mu (\text{平均})}{\sigma (\text{標準偏差})} $$
- 入ってきたデータ（ミニバッチ）に対して**標準化「平均を0,分散を1」**
- データから平均値を引くことで中心を0としている

In [ ]:
import tensorflow as tensorflow
from tensorflow.keras.layers import Input, Conv2D, Activation, Add, BatchNormalization


def residual_block(input_tensor, num_filters, strides=1):
    """
    入力と出力のサイズが変わらない、基本的な残差ブロック
    残差：Residual
    Output = f(Input)+Input
        ここでFは2回の畳み込みとReLU活性化

    Skip Connection
        残差分だけを学習することで、層が深くなっても勾配損失問題を防ぐ

    Conv
        畳み込みをすると通常は画面の端っこが削れて小さくなるが、padding='same'を指定して周りに余白をつける
        そうすると入力と出力のサイズが変わらなくなる

    Add()([x,shortcut])
        KerasのFunctional APIの書き方
            ➀足し算マシーン(Addレイヤー)の生成、ここでAddは単純に足し算をするだけなので()の中に複雑な設定が必要ない
            ➁足し算マシーンに[x,shortcut]というリストの形状にして足し算を実行する

    Strides
        畳み込みの移動幅を指定するパラメータ
        1なら通常通り1pxずつ、2なら2pxずつ移動する(1個飛ばし)ので、出力サイズが半分になる

    num_filters
        各num_filterは畳み込み層で検知した特徴マップを持っている
        例えば浅い層のフィルターではエッジ検出、深い層ではより複雑なパターンを検出する
            エッジ：隣り合う画素(ピクセル)が急激に変化している部分
    """
    shortcut = input_tensor

    # 畳み込み一回目
    x = Conv2D(num_filters, (3, 3), padding="same", strides=strides)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # 畳み込み二回目
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)

    if strides > 1 or input_tensor.shape[-1] != num_filters:
        """
        サイズが変わる、またはチャンネル数が変わるとき
        .shape[-1]は最後の要素、つまりchannnel
        """
        shortcut = Conv2D(num_filters, (1, 1), padding="same", strides=strides)(
            shortcut
        )
        shortcut = BatchNormalization()(shortcut)

    x = Add()([x, shortcut])
    x = Activation("relu")(x)
    return x

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense


def build_simple_resnet(input_shape=(128, 128, 3), num_classes=10):
    """
    ResNetの簡易モデルの作成
        画像分類タスクでは、画像の特徴の身を知りたいので畳み込み層のエンコードしか必要ない
        セグメンテーションは位置情報も必要なのでdecodeも必要になる
        ResNet18の18は層の数、50や152などがある

    Projection(射影)
            ➀入力の特徴マップのサイズを変換する
            ➁フィルター数(channel数)を変更する

    GlobalAveragePooling2D
        Strides=1は画像サイズを変えないまま特徴をしっかり抽出する

    """
    inputs = Input(input_shape)
    x = residual_block(inputs, num_filters=64, strides=1)

    x = residual_block(
        x, num_filters=64, strides=1
    )  # サイズを変えないまま特徴をしっかり見極める
    x = residual_block(x, num_filters=64, strides=1)

    x = residual_block(
        x, num_filters=128, strides=2
    )  # サイズを半分にする(Projection Block)
    x = residual_block(x, num_filters=128, strides=1)

    # --- 出口 (Output Layers) ---
    x = GlobalAveragePooling2D()(x)

    # 最終判定 (10クラス分類)
    outputs = Dense(num_classes, activation="softmax")(x)

    return Model(inputs, outputs)

### 1. GlobalAveragePooling2D　(GAP)
**位置情報は捨てて、特徴の強さだけを残す層**
> 一枚の特徴マップの全画素の平均を計算して、有効な数字であればその特徴が存在すると考える
  - 入力: $(Batch, 8, 8, 128)$ → 高さ8, 幅8, チャンネル128
  - 出力: $(Batch, 128)$ → 長さ128のベクトル

### 2. Dense （全結合層）
  - 最後のDense層のユニット数は、**分類したいクラス数**（EuroSATなら10個）と一致させる
  - 最後に`Softmax`関数を通すことで、出力を確率（合計すると100%）に変換

In [ ]:
import tensorflow_datasets as tfds
import os

# データを保存するディレクトリ (先ほど作った data フォルダを指定)
DATA_DIR = "data"

"""
tfds.load()でdatasetをダウンロードして読みこむ
    タプルのデータセットとinfoを返す
        eurosatのRGBバージョンを使用する
        as_supervised=True で(画像, ラベル)のタプルで返す
        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'] で学習・検証・テストに分割

"""
(train_ds, val_ds, test_ds), info = tfds.load(
    "eurosat/rgb",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    data_dir=DATA_DIR,
    as_supervised=True,
    with_info=True,
)

print("ダウンロードと読み込みが完了しました！")
print(f"クラス名: {info.features['label'].names}")
print(f"学習データ数: {len(train_ds)}")
print(f"検証データ数: {len(val_ds)}")
print(f"テストデータ数: {len(test_ds)}")

ダウンロードと読み込みが完了しました！
クラス名: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
学習データ数: 21600
検証データ数: 2700
テストデータ数: 2700


In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)

print(f"現在のポリシー: {policy.compute_dtype}")

現在のポリシー: float16


In [ ]:
import tensorflow as tf

# Setting constants
IMG_SIZE = 64
BATCH_SIZE = 256  # AI工房のRTX 3090 + VRAM 24GB


def preprocess_data(image, label):
    """
    画像データの前処理を行う関数
        1. サイズを確実に合わせる (リサイズ)
            - `tf.image.resize()`はTensorflowのimageモジュールの中にある関数
            - interpolation 補間：いい感じに新しい色を計算して埋める
            > 有限要素法のバイリニア補間
                すべての画素に対して、周囲の整数格子点の値を使って線形補間を行う方法

        2. 正規化 (Normalization)
            - 画像のRGB各チャンネルの値を0から1の範囲にスケーリングする
                - 正規化しないと計算量が膨大になる

        .take(n)
            - データセットから最初のn個の要素を取得するメソッド
    """
    image = tf.image.resize(
        image, (IMG_SIZE, IMG_SIZE)
    )  # 1.サイズを確実に合わせる(リサイズ)
    image = tf.cast(image, tf.float32) / 255.0  # 2.正規化(Normalization)
    return image, label


# --- パイプラインの構築 ---
AUTOTUNE = tf.data.AUTOTUNE
train_batches = (
    train_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .shuffle(buffer_size=1000)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

val_batches = (
    val_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

# 確認
for img_batch, label_batch in train_batches.take(1):
    print(f"Image Batch Shape: {img_batch.shape}")
    print(f"Label Batch Shape: {label_batch.shape}")

    # Image: (32, 64, 64, 3) -> 32枚, 64x64ピクセル, 3チャンネル(RGB)
    # Label: (32,) -> 32個の正解ラベル

Image Batch Shape: (32, 64, 64, 3)
Label Batch Shape: (32,)


### データパイプライン DataPipeLine
> データパイプラインとは、データをストレージから読み出し、計算モデルが処理可能なテンソルへと変換し、GPUやTPUへ供給する一連の処理工程、およびそれを実装したソフトウェアアーキテクチャを指す。

In [ ]:
# input_shapeは (64, 64, 3), クラス数は 10
model = build_simple_resnet(input_shape=(64, 64, 3), num_classes=10)

# --- コンパイル (学習ルールの設定) ---
model.compile(
    # AIの「間違いの修正方法」を指定します。Adamは最も一般的で優秀な修正担当者です。
    optimizer="adam",  # Optimizer (最適化アルゴリズム): 'adam'
    # Loss (損失関数): 'SparseCategoricalCrossentropy'
    # from_logits=True は、AIの生の出力を確率に変換してから計算しろ、という指示です。
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    # Metrics:'accuracy'
    metrics=["accuracy"],
)

# モデルの設計図確認
# model.summary()

In [ ]:
# --- 学習の実行 ---
history = model.fit(train_batches, validation_data=val_batches, epochs=20)

Epoch 1/5


KeyboardInterrupt: 

###　Result
- 自分のPCのCPU,Core i7だと90分かかった![Result of CPU Learning](images/122101.png)
<br>
- AI工房のGPUサーバーを使うと40秒程度で5epoch終了->**135倍!!のスピード**![Result of GPU Learning](images/122102.png)

In [ ]:
import matplotlib.pyplot as plt


def plot_learning_curve(history):
    """
    学習曲線をプロットし、何エポックで頭打ちになるかを視覚化する
    """
    epochs = range(1, len(history.history["accuracy"]) + 1)
    train_acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]

    # 頭打ちポイントの検出（検証精度の最大値）
    best_epoch = val_acc.index(max(val_acc)) + 1
    best_val_acc = max(val_acc)

    # プロット
    fig, ax = plt.subplots(figsize=(10, 6))

    ax.plot(
        epochs,
        train_acc,
        "b-",
        label="Training Accuracy",
        linewidth=2,
        marker="o",
        markersize=5,
    )
    ax.plot(
        epochs,
        val_acc,
        "r-",
        label="Validation Accuracy",
        linewidth=2,
        marker="s",
        markersize=5,
    )

    # 最高精度のポイントをマーク
    ax.axvline(
        x=best_epoch,
        color="green",
        linestyle="--",
        alpha=0.7,
        label=f"Best Epoch: {best_epoch}",
    )
    ax.scatter([best_epoch], [best_val_acc], color="green", s=200, zorder=5, marker="★")
    ax.annotate(
        f"Best: {best_val_acc:.4f}\n(Epoch {best_epoch})",
        xy=(best_epoch, best_val_acc),
        xytext=(best_epoch + 1, best_val_acc - 0.05),
        fontsize=12,
        fontweight="bold",
        arrowprops=dict(arrowstyle="->", color="green"),
    )

    # 頭打ち領域を薄く塗る（最高精度以降）
    if best_epoch < len(epochs):
        ax.axvspan(
            best_epoch, len(epochs), alpha=0.1, color="gray", label="Plateau Region"
        )

    ax.set_xlabel("Epoch", fontsize=14)
    ax.set_ylabel("Accuracy", fontsize=14)
    ax.set_title("Learning Curve: Training vs Validation Accuracy", fontsize=16)
    ax.legend(loc="lower right", fontsize=11)
    ax.grid(True, linestyle=":", alpha=0.6)
    ax.set_xlim(0.5, len(epochs) + 0.5)
    ax.set_ylim(0, 1.05)

    # X軸を整数エポックで表示
    ax.set_xticks(epochs)

    plt.tight_layout()
    plt.savefig("learning_curve.png", dpi=300, bbox_inches="tight")
    print(f"学習曲線を保存しました: learning_curve.png")
    plt.show()

    # 分析結果の出力
    print("\n" + "=" * 50)
    print("【学習曲線の分析】")
    print("=" * 50)
    print(f"• 最高検証精度: {best_val_acc:.4f} (Epoch {best_epoch})")
    print(f"• 最終訓練精度: {train_acc[-1]:.4f}")
    print(f"• 最終検証精度: {val_acc[-1]:.4f}")

    # 過学習の判定
    gap = train_acc[-1] - val_acc[-1]
    if gap > 0.1:
        print(f"• ⚠️ 過学習の傾向あり（Train-Val差: {gap:.4f}）")
    elif val_acc[-1] < best_val_acc - 0.01:
        print(f"• 🟡 Epoch {best_epoch} 以降は精度が低下傾向")
    else:
        print(f"• 🟢 安定した学習が継続")

    # 頭打ちの判定
    if best_epoch <= len(epochs) * 0.5:
        print(f"• 📊 Epoch {best_epoch} で頭打ち → エポック数を減らしても良い可能性")
    elif best_epoch >= len(epochs) - 2:
        print(f"• 📊 まだ改善の余地あり → エポック数を増やすと精度向上の可能性")


# 実行
plot_learning_curve(history)

In [ ]:
# --- テストデータの準備と評価 ---

# テストデータにも同様の前処理（リサイズ・正規化）とバッチ化を適用
test_batches = (
    test_ds.map(preprocess_data, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

# モデルの評価 (未学習データでの性能確認)
print("テストデータで評価を実行します...")
test_loss, test_acc = model.evaluate(test_batches)

print(f"\nテストデータの正解率: {test_acc * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# クラス名を取得
class_names = info.features["label"].names


def plot_predictions(dataset, model, num_images=9):
    """
    テストデータに対して推論を行い、画像と予測結果を表示する関数
    """
    plt.figure(figsize=(12, 12))

    # データセットから1バッチ(32枚)だけ取り出す
    for images, labels in dataset.take(1):
        # 推論の実行 (確率が出力される)
        predictions = model.predict(images)
        # 最も確率が高いクラスのインデックスを取得
        pred_indices = np.argmax(predictions, axis=1)

        # 指定枚数分だけ表示
        for i in range(min(num_images, len(images))):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i])

            # ラベル名の取得
            true_label = class_names[labels[i]]
            pred_label = class_names[pred_indices[i]]
            confidence = 100 * np.max(predictions[i])

            # 正解なら青、不正解なら赤でタイトルを表示
            color = "blue" if labels[i] == pred_indices[i] else "red"

            plt.title(
                f"Pred: {pred_label} ({confidence:.1f}%)\nTrue: {true_label}",
                color=color,
            )
            plt.axis("off")

    plt.tight_layout()
    plt.show()


# 可視化の実行
print("推論結果の可視化:")
plot_predictions(test_batches, model)

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm

# --- 分析用データの準備 ---
print("詳細なエラー分析を実行します...")

# 全テストデータの予測と正解ラベルの取得
all_images = []
all_labels = []

# データセットから全データを取得
for img_batch, label_batch in test_batches:
    all_images.append(img_batch.numpy())
    all_labels.append(label_batch.numpy())

x_test = np.concatenate(all_images)
y_test = np.concatenate(all_labels)

# 推論実行 (バッチ処理しているので高速)
predictions = model.predict(x_test, verbose=0)
pred_labels = np.argmax(predictions, axis=1)
max_probs = np.max(predictions, axis=1)

# --- 1. 混同行列 (Confusion Matrix) ---
conf_matrix = confusion_matrix(y_test, pred_labels)

# --- 2. 「自信満々に間違えた」データの抽出 ---
incorrect_indices = np.where(pred_labels != y_test)[0]
# 予測確率(自信)が高い順にソートしてトップを取得
sorted_incorrect_indices = incorrect_indices[
    np.argsort(max_probs[incorrect_indices])[::-1]
]


# --- 3. Grad-CAM (判断根拠の可視化) ---
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Grad-CAM ヒートマップを生成する関数

    """
    if len(img_array.shape) == 3:
        img_array = np.expand_dims(img_array, axis=0)

    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


def find_target_layer(model):
    for layer in reversed(model.layers):
        # layer.output_shape は古いKerasの仕様です。代わりに layer.output.shape を使用します。
        try:
            if hasattr(layer, "output") and len(layer.output.shape) == 4:
                return layer.name
        except AttributeError:
            continue
    return None


target_layer_name = find_target_layer(model)

# --- 可視化プロット ---
plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 0.5, 0.5])

# 左側: 混同行列
ax_cm = plt.subplot(gs[:, :2])
sns.heatmap(
    conf_matrix,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
    ax=ax_cm,
)
ax_cm.set_title("Confusion Matrix", fontsize=16)
ax_cm.set_ylabel("True Label", fontsize=14)
ax_cm.set_xlabel("Predicted Label", fontsize=14)

# 右側: 自信満々に間違えた画像 Top 4
num_display = 4
for i in range(min(num_display, len(sorted_incorrect_indices))):
    idx = sorted_incorrect_indices[i]
    img = x_test[idx]
    true_lb = class_names[y_test[idx]]
    pred_lb = class_names[pred_labels[idx]]
    conf = max_probs[idx]

    # Grad-CAM ヒートマップ
    heatmap = make_gradcam_heatmap(
        img, model, target_layer_name, pred_index=pred_labels[idx]
    )

    # リサイズと重ね合わせ表示
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (IMG_SIZE, IMG_SIZE))
    heatmap_resized = tf.squeeze(heatmap_resized).numpy()

    # グリッド配置 (右半分の領域を使う)
    row = i // 2
    col = 2 + (i % 2)
    ax_img = plt.subplot(gs[row, col])

    ax_img.imshow(img)
    ax_img.imshow(heatmap_resized, alpha=0.5, cmap="jet")

    # 枠線の色 (赤: 間違い)
    for spine in ax_img.spines.values():
        spine.set_edgecolor("red")
        spine.set_linewidth(3)

    ax_img.set_title(
        f"True: {true_lb}\nPred: {pred_lb}\nConf: {conf:.1%}",
        fontsize=12,
        color="red",
        fontweight="bold",
    )
    ax_img.set_xticks([])
    ax_img.set_yticks([])

plt.suptitle(
    f"Error Analysis & Grad-CAM (Target Layer: {target_layer_name})", fontsize=20
)
plt.tight_layout()
plt.show()

# 考察コメントの例示
print("\n=== 考察コメントの例 ===")
print(
    "Grad-CAMのヒートマップ（赤色部分）を確認することで、モデルが画像のどこを見て判断したかが分かります。"
)
print(
    "これを用いて、『形ではなく色だけで判断してしまっている』などの誤答原因を分析できます。"
)

In [ ]:
import matplotlib.pyplot as plt


def plot_color_histograms(dataset, class_names, target_classes):
    """
    指定した2つのクラスの「色の分布」を比較する
    target_classes: ['SeaLake', 'Pasture'] のように指定
    """
    colors = ["Red", "Green", "Blue"]
    plt.figure(figsize=(15, 5))

    # データを少し集める
    target_images = {name: [] for name in target_classes}

    for img, label in dataset.take(200):  # 200枚くらいチェック
        name = class_names[label.numpy()]
        if name in target_classes:
            target_images[name].append(img.numpy())

    # 各チャンネルごとにヒストグラムを描画
    for i, color in enumerate(colors):
        plt.subplot(1, 3, i + 1)
        for name in target_classes:
            if len(target_images[name]) == 0:
                continue

            # 全画像のピクセル値をまとめる
            pixels = np.concatenate(target_images[name])
            pixel_values = pixels[:, :, i].flatten()  # 1次元に伸ばす

            plt.hist(pixel_values, bins=50, alpha=0.5, label=name, density=True)

        plt.title(f"{color} Channel Distribution")
        plt.legend()
        plt.xlabel("Pixel Value (0-1)")

    plt.show()


# 実行
plot_color_histograms(test_ds, info.features["label"].names, ["SeaLake", "Pasture"])

### RGB 3D分布による誤分類原因の分析

ResNetの誤分類がRGB（色）に起因するかを確認するため、以下の分析を行います。

#### 分析手法
1. **3D散布図**: 各画像のRGB平均値をプロット
2. **定量分析**: 誤分類サンプルが「元クラス」と「混同先クラス」どちらに近いか計算
3. **統計的検定**: 正解/誤分類サンプル間でRGB特性に有意差があるか検証
4. **RGB分散分析**: 画像内の色のばらつきが誤分類に関係するか調査

#### 考察の観点

| 観察結果 | 解釈 |
|---------|------|
| 誤分類サンプルが混同先クラスの領域にある | → RGBの類似性が原因 |
| 誤分類サンプルが元クラス内に散在している | → RGB以外の要因（テクスチャ、エッジ、形状）が原因 |
| 誤分類サンプルに特定のRGB傾向がない | → RGBは分類に寄与していない可能性 |

#### RGB平均の限界
- 画像全体の平均値は**局所的な特徴を消してしまう**
- 同じRGB平均でも、テクスチャや空間パターンは異なりうる
- 衛星画像では、物体の**配置**や**繰り返しパターン**が重要なことが多い

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_rgb_3d_interactive(x_test, y_test, pred_labels, class_names):
    """
    RGB平均値を3D空間にプロットし、正解/誤分類を可視化する関数

    Parameters:
    -----------
    x_test : 正規化済みテスト画像 (0-1)
    y_test : 正解ラベル
    pred_labels : 予測ラベル
    class_names : クラス名リスト
    """

    # ==========================================
    # 1. 各画像のRGB平均値を計算
    # ==========================================
    print("各画像のRGB平均値を計算中...")

    # 画像ごとのRGB平均 (0-255スケールに戻す)
    rgb_means = np.mean(x_test, axis=(1, 2)) * 255  # shape: (N, 3)

    # 正解/誤分類のマスク
    correct_mask = pred_labels == y_test
    incorrect_mask = ~correct_mask

    print(f"正解数: {np.sum(correct_mask)}, 誤分類数: {np.sum(incorrect_mask)}")

    # ==========================================
    # 2. Plotlyでインタラクティブ3Dプロット
    # ==========================================
    fig = go.Figure()

    # カラーマップ (10クラス用)
    colors = [
        "#1f77b4",
        "#ff7f0e",
        "#2ca02c",
        "#d62728",
        "#9467bd",
        "#8c564b",
        "#e377c2",
        "#7f7f7f",
        "#bcbd22",
        "#17becf",
    ]

    # --- 正解サンプル (薄く表示) ---
    for i, class_name in enumerate(class_names):
        mask = (y_test == i) & correct_mask
        if np.sum(mask) == 0:
            continue

        data = rgb_means[mask]
        fig.add_trace(
            go.Scatter3d(
                x=data[:, 0],
                y=data[:, 1],
                z=data[:, 2],
                mode="markers",
                marker=dict(size=3, color=colors[i], opacity=0.3),
                name=f"{class_name} (正解)",
                legendgroup=class_name,
                hovertemplate=f"<b>{class_name}</b><br>R: %{{x:.1f}}<br>G: %{{y:.1f}}<br>B: %{{z:.1f}}<extra></extra>",
            )
        )

    # --- 誤分類サンプル (大きく目立たせる) ---
    for i, class_name in enumerate(class_names):
        # このクラスに属する画像で誤分類されたもの
        mask = (y_test == i) & incorrect_mask
        if np.sum(mask) == 0:
            continue

        data = rgb_means[mask]
        pred_class_indices = pred_labels[mask]

        # ホバーテキストに予測クラスも表示
        hover_texts = [
            f"<b>True: {class_name}</b><br>Pred: {class_names[p]}<br>R: {r:.1f}, G: {g:.1f}, B: {b:.1f}"
            for (r, g, b), p in zip(data, pred_class_indices)
        ]

        fig.add_trace(
            go.Scatter3d(
                x=data[:, 0],
                y=data[:, 1],
                z=data[:, 2],
                mode="markers",
                marker=dict(
                    size=8,
                    color=colors[i],
                    opacity=1.0,
                    symbol="x",  # ×マーカーで誤分類を強調
                    line=dict(width=2, color="black"),
                ),
                name=f"{class_name} (誤分類)",
                legendgroup=class_name,
                hovertemplate="%{text}<extra></extra>",
                text=hover_texts,
            )
        )

    # --- 各クラスの重心を表示 ---
    for i, class_name in enumerate(class_names):
        mask = y_test == i
        if np.sum(mask) == 0:
            continue

        center = np.mean(rgb_means[mask], axis=0)
        fig.add_trace(
            go.Scatter3d(
                x=[center[0]],
                y=[center[1]],
                z=[center[2]],
                mode="markers+text",
                marker=dict(
                    size=15,
                    color=colors[i],
                    symbol="diamond",
                    line=dict(width=3, color="white"),
                ),
                text=[class_name],
                textposition="top center",
                textfont=dict(size=12, color="black"),
                name=f"{class_name} 重心",
                legendgroup=class_name,
                showlegend=False,
                hovertemplate=f"<b>{class_name} 重心</b><br>R: {center[0]:.1f}<br>G: {center[1]:.1f}<br>B: {center[2]:.1f}<extra></extra>",
            )
        )

    # レイアウト設定
    fig.update_layout(
        title=dict(
            text="RGB Distribution - 正解 vs 誤分類<br><sub>×マーカー = 誤分類サンプル</sub>",
            font=dict(size=20),
        ),
        scene=dict(
            xaxis_title="Red Mean",
            yaxis_title="Green Mean",
            zaxis_title="Blue Mean",
            xaxis=dict(range=[0, 255]),
            yaxis=dict(range=[0, 255]),
            zaxis=dict(range=[0, 255]),
            aspectmode="cube",
        ),
        width=1000,
        height=800,
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.02, font=dict(size=10)),
        margin=dict(r=200),
    )

    fig.show()

    # ==========================================
    # 3. 誤分類の統計情報
    # ==========================================
    print("\n=== 誤分類のRGB分析 ===")

    for i, class_name in enumerate(class_names):
        correct = rgb_means[(y_test == i) & correct_mask]
        incorrect = rgb_means[(y_test == i) & incorrect_mask]

        if len(incorrect) > 0 and len(correct) > 0:
            correct_mean = np.mean(correct, axis=0)
            incorrect_mean = np.mean(incorrect, axis=0)
            diff = incorrect_mean - correct_mean

            print(f"\n【{class_name}】 誤分類: {len(incorrect)}件")
            print(
                f"  正解サンプルの平均RGB: R={correct_mean[0]:.1f}, G={correct_mean[1]:.1f}, B={correct_mean[2]:.1f}"
            )
            print(
                f"  誤分類サンプルの平均RGB: R={incorrect_mean[0]:.1f}, G={incorrect_mean[1]:.1f}, B={incorrect_mean[2]:.1f}"
            )
            print(
                f"  差分 (誤分類-正解): ΔR={diff[0]:+.1f}, ΔG={diff[1]:+.1f}, ΔB={diff[2]:+.1f}"
            )

    return rgb_means, correct_mask, incorrect_mask


# 実行 (前のセルで x_test, y_test, pred_labels が定義済みの前提)
print("3D RGB分布をプロットします...")
rgb_means, correct_mask, incorrect_mask = plot_rgb_3d_interactive(
    x_test, y_test, pred_labels, class_names
)

In [ ]:
def plot_confusion_pairs_rgb(x_test, y_test, pred_labels, class_names, top_n=5):
    """
    誤分類が多いクラスペアのRGB分布を詳細比較する
    """
    from collections import Counter
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    from scipy import stats

    # 誤分類ペアをカウント
    incorrect_mask = pred_labels != y_test
    pairs = [(y_test[i], pred_labels[i]) for i in np.where(incorrect_mask)[0]]
    pair_counts = Counter(pairs)

    print("=== 誤分類ペア Top {} ===".format(top_n))
    for (true_idx, pred_idx), count in pair_counts.most_common(top_n):
        print(f"  {class_names[true_idx]} → {class_names[pred_idx]}: {count}件")

    if len(pair_counts) == 0:
        print("誤分類がありません！")
        return

    top_pairs = pair_counts.most_common(min(top_n, 4))

    # RGB平均値
    rgb_means = np.mean(x_test, axis=(1, 2)) * 255

    # サブプロット作成 (2x2)
    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[
            [{"type": "scatter3d"}, {"type": "scatter3d"}],
            [{"type": "scatter3d"}, {"type": "scatter3d"}],
        ],
        subplot_titles=[
            f"{class_names[t]} ↔ {class_names[p]} ({c}件)" for (t, p), c in top_pairs
        ]
        + [""] * (4 - len(top_pairs)),
    )

    colors_correct = ["#2196F3", "#4CAF50"]
    colors_incorrect = ["#F44336", "#FF9800"]

    # ==========================================
    # 定量分析の準備
    # ==========================================
    analysis_results = []

    for idx, ((true_idx, pred_idx), count) in enumerate(top_pairs):
        row = idx // 2 + 1
        col = idx % 2 + 1

        true_name = class_names[true_idx]
        pred_name = class_names[pred_idx]

        true_class_mask = y_test == true_idx
        true_correct = true_class_mask & (pred_labels == y_test)
        true_wrong = true_class_mask & (pred_labels == pred_idx)

        pred_class_mask = y_test == pred_idx
        pred_correct = pred_class_mask & (pred_labels == y_test)

        # --- 定量分析 ---
        correct_data = rgb_means[true_correct]
        wrong_data = rgb_means[true_wrong]
        pred_class_data = rgb_means[pred_correct]

        if len(wrong_data) > 0 and len(correct_data) > 0 and len(pred_class_data) > 0:
            # 各クラスの重心
            correct_center = np.mean(correct_data, axis=0)
            wrong_center = np.mean(wrong_data, axis=0)
            pred_center = np.mean(pred_class_data, axis=0)

            # 誤分類サンプルが「元クラス重心」と「混同先クラス重心」のどちらに近いか
            dist_to_own = np.linalg.norm(wrong_center - correct_center)
            dist_to_pred = np.linalg.norm(wrong_center - pred_center)

            # 誤分類サンプルの分散（散らばり具合）
            wrong_variance = np.mean(np.var(wrong_data, axis=0))
            correct_variance = np.mean(np.var(correct_data, axis=0))

            # クラス間の重複度（重心間距離 / 分散）
            class_separation = np.linalg.norm(correct_center - pred_center)

            analysis_results.append(
                {
                    "pair": f"{true_name} → {pred_name}",
                    "count": count,
                    "dist_to_own": dist_to_own,
                    "dist_to_pred": dist_to_pred,
                    "closer_to": "pred_class"
                    if dist_to_pred < dist_to_own
                    else "own_class",
                    "class_separation": class_separation,
                    "wrong_variance": wrong_variance,
                    "correct_variance": correct_variance,
                }
            )

        # プロット
        if len(correct_data) > 0:
            fig.add_trace(
                go.Scatter3d(
                    x=correct_data[:, 0],
                    y=correct_data[:, 1],
                    z=correct_data[:, 2],
                    mode="markers",
                    marker=dict(size=4, color=colors_correct[0], opacity=0.5),
                    name=f"{true_name} (正解)",
                    showlegend=(idx == 0),
                ),
                row=row,
                col=col,
            )

        if len(wrong_data) > 0:
            fig.add_trace(
                go.Scatter3d(
                    x=wrong_data[:, 0],
                    y=wrong_data[:, 1],
                    z=wrong_data[:, 2],
                    mode="markers",
                    marker=dict(
                        size=8, color=colors_incorrect[0], opacity=1.0, symbol="x"
                    ),
                    name=f"{true_name}→{pred_name} (誤分類)",
                    showlegend=(idx == 0),
                ),
                row=row,
                col=col,
            )

        if len(pred_class_data) > 0:
            fig.add_trace(
                go.Scatter3d(
                    x=pred_class_data[:, 0],
                    y=pred_class_data[:, 1],
                    z=pred_class_data[:, 2],
                    mode="markers",
                    marker=dict(size=4, color=colors_correct[1], opacity=0.5),
                    name=f"{pred_name} (正解)",
                    showlegend=(idx == 0),
                ),
                row=row,
                col=col,
            )

    fig.update_layout(
        title="誤分類ペア別 RGB分布比較",
        height=900,
        width=1100,
    )

    for i in range(1, 5):
        fig.update_scenes(
            dict(xaxis_title="R", yaxis_title="G", zaxis_title="B", aspectmode="cube"),
            row=(i - 1) // 2 + 1,
            col=(i - 1) % 2 + 1,
        )

    fig.show()

    # ==========================================
    # 定量的な考察の出力
    # ==========================================
    print("\n" + "=" * 60)
    print("【定量分析による考察】")
    print("=" * 60)

    for result in analysis_results:
        print(f"\n■ {result['pair']} ({result['count']}件)")
        print(
            f"  ├─ 誤分類サンプル → 元クラス重心までの距離: {result['dist_to_own']:.2f}"
        )
        print(
            f"  ├─ 誤分類サンプル → 混同先クラス重心までの距離: {result['dist_to_pred']:.2f}"
        )
        print(f"  ├─ 2クラス間の重心距離 (分離度): {result['class_separation']:.2f}")
        print(
            f"  └─ 誤分類サンプルの分散: {result['wrong_variance']:.2f} (正解: {result['correct_variance']:.2f})"
        )

        # 考察の自動判定
        if result["dist_to_pred"] < result["dist_to_own"]:
            print(
                f"  → 🔴 誤分類サンプルは混同先クラスに近い → RGBが誤分類の原因の可能性"
            )
        elif result["dist_to_own"] < result["class_separation"] * 0.3:
            print(
                f"  → 🟢 誤分類サンプルは元クラス内に留まっている → RGB以外の要因が原因"
            )
        else:
            print(f"  → 🟡 誤分類サンプルは両クラスの中間領域 → クラス境界が曖昧")

    # 全体的な考察
    print("\n" + "=" * 60)
    print("【RGB空間での分析まとめ】")
    print("=" * 60)

    rgb_caused = sum(1 for r in analysis_results if r["closer_to"] == "pred_class")
    total = len(analysis_results)

    if total > 0:
        rgb_ratio = rgb_caused / total
        print(
            f"\n• 誤分類がRGBに起因する可能性が高いペア: {rgb_caused}/{total} ({rgb_ratio * 100:.0f}%)"
        )

        if rgb_ratio > 0.5:
            print(
                "\n📊 結論: 誤分類の多くはRGB（色）の類似性に起因している可能性が高い"
            )
            print("   → 色が似ているクラス間での誤分類が多い")
            print("   → 対策: 色以外の特徴（テクスチャ、エッジ）を強調する前処理や、")
            print("           Data Augmentation（色変換）が有効かもしれない")
        else:
            print("\n📊 結論: 誤分類はRGBの類似性だけでは説明できない")
            print("   → 誤分類サンプルは元のクラスと同じRGB領域に存在している")
            print("   → RGB平均値だけでは捉えられない特徴が分類に重要")
            print("   → 考えられる要因:")
            print("     • テクスチャ（繰り返しパターン）")
            print("     • エッジ・輪郭の形状")
            print("     • 空間的な色の配置（画像の上下左右での色の違い）")
            print("     • 局所的な特徴（RGB平均では消えてしまう）")


# 実行
plot_confusion_pairs_rgb(x_test, y_test, pred_labels, class_names)

In [ ]:
def analyze_rgb_distribution_detail(x_test, y_test, pred_labels, class_names):
    """
    より詳細なRGB分析：分散、ヒストグラム、空間的分布
    """
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    incorrect_mask = pred_labels != y_test
    correct_mask = ~incorrect_mask

    # RGB平均と分散
    rgb_means = np.mean(x_test, axis=(1, 2)) * 255
    rgb_stds = np.std(x_test, axis=(1, 2)) * 255  # 画像内のRGB分散

    # ==========================================
    # 1. 誤分類vs正解の RGB分散比較
    # ==========================================
    fig = make_subplots(
        rows=2,
        cols=2,
        specs=[
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}],
        ],
        subplot_titles=[
            "Red: 平均 vs 分散",
            "Green: 平均 vs 分散",
            "Blue: 平均 vs 分散",
            "RGB分散の合計 (正解 vs 誤分類)",
        ],
    )

    channel_names = ["Red", "Green", "Blue"]

    for ch, name in enumerate(channel_names):
        row = (ch // 2) + 1
        col = (ch % 2) + 1

        # 正解サンプル
        fig.add_trace(
            go.Scatter(
                x=rgb_means[correct_mask, ch],
                y=rgb_stds[correct_mask, ch],
                mode="markers",
                marker=dict(size=3, color="blue", opacity=0.3),
                name="正解",
                showlegend=(ch == 0),
            ),
            row=row,
            col=col,
        )

        # 誤分類サンプル
        fig.add_trace(
            go.Scatter(
                x=rgb_means[incorrect_mask, ch],
                y=rgb_stds[incorrect_mask, ch],
                mode="markers",
                marker=dict(size=6, color="red", opacity=0.7),
                name="誤分類",
                showlegend=(ch == 0),
            ),
            row=row,
            col=col,
        )

    # RGB分散の合計
    total_std_correct = np.sum(rgb_stds[correct_mask], axis=1)
    total_std_incorrect = np.sum(rgb_stds[incorrect_mask], axis=1)

    fig.add_trace(
        go.Histogram(
            x=total_std_correct,
            name="正解",
            opacity=0.6,
            marker_color="blue",
            showlegend=False,
        ),
        row=2,
        col=2,
    )

    fig.add_trace(
        go.Histogram(
            x=total_std_incorrect,
            name="誤分類",
            opacity=0.6,
            marker_color="red",
            showlegend=False,
        ),
        row=2,
        col=2,
    )

    fig.update_layout(
        title="RGB平均 vs RGB分散（画像内のばらつき）",
        height=700,
        width=900,
        barmode="overlay",
    )

    fig.show()

    # ==========================================
    # 2. 統計的検定
    # ==========================================
    from scipy import stats

    print("\n" + "=" * 60)
    print("【統計的分析】誤分類サンプルの特徴")
    print("=" * 60)

    # RGB平均の比較
    print("\n■ RGB平均値の比較 (t検定)")
    for ch, name in enumerate(channel_names):
        correct_vals = rgb_means[correct_mask, ch]
        incorrect_vals = rgb_means[incorrect_mask, ch]
        t_stat, p_val = stats.ttest_ind(correct_vals, incorrect_vals)
        sig = "有意差あり ✓" if p_val < 0.05 else "有意差なし"
        print(
            f"  {name}: 正解={np.mean(correct_vals):.1f}±{np.std(correct_vals):.1f}, "
            f"誤分類={np.mean(incorrect_vals):.1f}±{np.std(incorrect_vals):.1f} (p={p_val:.4f}) {sig}"
        )

    # RGB分散の比較
    print("\n■ RGB分散（画像内ばらつき）の比較")
    for ch, name in enumerate(channel_names):
        correct_vals = rgb_stds[correct_mask, ch]
        incorrect_vals = rgb_stds[incorrect_mask, ch]
        t_stat, p_val = stats.ttest_ind(correct_vals, incorrect_vals)
        sig = "有意差あり ✓" if p_val < 0.05 else "有意差なし"
        diff = np.mean(incorrect_vals) - np.mean(correct_vals)
        direction = "高い↑" if diff > 0 else "低い↓"
        print(
            f"  {name}: 誤分類は正解より分散が{direction} (差={diff:+.2f}, p={p_val:.4f}) {sig}"
        )

    # ==========================================
    # 3. 空間的なRGB分布（上下左右）
    # ==========================================
    print("\n■ 空間的なRGB分布の分析")

    # 画像を上半分と下半分に分けて比較
    h = x_test.shape[1] // 2
    top_mean = np.mean(x_test[:, :h, :, :], axis=(1, 2)) * 255
    bottom_mean = np.mean(x_test[:, h:, :, :], axis=(1, 2)) * 255
    vertical_diff = top_mean - bottom_mean  # 上下の色の違い

    # 誤分類サンプルは上下の色差が大きい/小さい？
    correct_v_diff = np.mean(np.abs(vertical_diff[correct_mask]), axis=1)
    incorrect_v_diff = np.mean(np.abs(vertical_diff[incorrect_mask]), axis=1)

    t_stat, p_val = stats.ttest_ind(correct_v_diff, incorrect_v_diff)
    print(
        f"  上下の色差: 正解={np.mean(correct_v_diff):.2f}, 誤分類={np.mean(incorrect_v_diff):.2f} (p={p_val:.4f})"
    )

    # ==========================================
    # 4. 考察のまとめ
    # ==========================================
    print("\n" + "=" * 60)
    print("【考察】")
    print("=" * 60)

    # RGB平均に有意差があるか
    rgb_mean_significant = False
    for ch in range(3):
        _, p_val = stats.ttest_ind(
            rgb_means[correct_mask, ch], rgb_means[incorrect_mask, ch]
        )
        if p_val < 0.05:
            rgb_mean_significant = True
            break

    # RGB分散に有意差があるか
    rgb_std_significant = False
    for ch in range(3):
        _, p_val = stats.ttest_ind(
            rgb_stds[correct_mask, ch], rgb_stds[incorrect_mask, ch]
        )
        if p_val < 0.05:
            rgb_std_significant = True
            break

    if not rgb_mean_significant and not rgb_std_significant:
        print("\n🟢 誤分類サンプルと正解サンプルのRGB特性に統計的な有意差がない")
        print("   → RGBの平均値・分散だけでは誤分類を説明できない")
        print(
            "   → モデルはRGB以外の特徴（テクスチャ、形状、エッジ等）で判断している可能性が高い"
        )
    elif rgb_mean_significant:
        print("\n🔴 誤分類サンプルはRGB平均値に特徴がある")
        print("   → 特定の色域で誤分類が起きやすい")
    elif rgb_std_significant:
        print("\n🟡 誤分類サンプルはRGB分散（画像内のばらつき）に特徴がある")
        print("   → 均一な色の画像 or 色が複雑な画像で誤分類しやすい可能性")


# 実行
analyze_rgb_distribution_detail(x_test, y_test, pred_labels, class_names)

In [ ]:
def compare_water_vs_vegetation(x_test, y_test, pred_labels, class_names):
    """
    水域（SeaLake, River）と緑系植生（Pasture, AnnualCrop）の色分布を比較
    → 水域に緑色が存在すると、植生と誤分類されやすいことを検証
    """
    import matplotlib.pyplot as plt
    from scipy import stats

    # 比較するクラスペア（水域 vs 緑系植生）
    pairs_to_compare = [
        ("SeaLake", "Pasture", "湖沼の緑 → 牧草地と誤認"),
        ("River", "AnnualCrop", "河川の緑 → 作物と誤認"),
    ]

    # クラス名からインデックスを取得
    class_to_idx = {name: i for i, name in enumerate(class_names)}

    # RGB平均値
    rgb_means = np.mean(x_test, axis=(1, 2)) * 255

    # 正解/誤分類のマスク
    correct_mask = pred_labels == y_test
    incorrect_mask = ~correct_mask

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    channel_names = ["Red", "Green", "Blue"]

    print("=" * 70)
    print("【水域の緑色成分が植生との誤分類を引き起こす可能性の検証】")
    print("=" * 70)
    print("\n仮説: SeaLakeやRiverに藻類等の緑色が存在すると、")
    print("      Pasture（牧草地）やAnnualCrop（作物）と誤分類されやすい")

    for row, (water_class, veg_class, description) in enumerate(pairs_to_compare):
        if water_class not in class_to_idx or veg_class not in class_to_idx:
            print(f"⚠️ {water_class} または {veg_class} がデータセットに存在しません")
            continue

        idx_water = class_to_idx[water_class]
        idx_veg = class_to_idx[veg_class]

        # 水域クラスのデータ
        mask_water_all = y_test == idx_water
        mask_water_correct = mask_water_all & correct_mask
        mask_water_wrong_to_veg = mask_water_all & (pred_labels == idx_veg)

        # 植生クラスのデータ
        mask_veg = y_test == idx_veg

        rgb_water_correct = rgb_means[mask_water_correct]  # 正しく分類された水域
        rgb_water_wrong = rgb_means[mask_water_wrong_to_veg]  # 植生と誤分類された水域
        rgb_veg = rgb_means[mask_veg]  # 植生クラス全体

        print(f"\n{'=' * 70}")
        print(f"■ {description}")
        print(f"  {water_class}（水域）→ {veg_class}（植生）への誤分類")
        print(f"  サンプル数:")
        print(f"    - {water_class} 正解: {len(rgb_water_correct)}件")
        print(f"    - {water_class} → {veg_class} 誤分類: {len(rgb_water_wrong)}件")
        print(f"    - {veg_class} 全体: {len(rgb_veg)}件")
        print("=" * 70)

        # --- 1. 各チャンネルのヒストグラム比較 ---
        for ch, ch_name in enumerate(channel_names):
            ax = axes[row, ch]

            # 正しく分類された水域（青色）
            if len(rgb_water_correct) > 0:
                ax.hist(
                    rgb_water_correct[:, ch],
                    bins=30,
                    alpha=0.5,
                    label=f"{water_class} (正解)",
                    color="#2196F3",
                    density=True,
                )

            # 植生と誤分類された水域（赤色、強調）
            if len(rgb_water_wrong) > 0:
                ax.hist(
                    rgb_water_wrong[:, ch],
                    bins=20,
                    alpha=0.8,
                    label=f"{water_class}→{veg_class} (誤分類)",
                    color="#F44336",
                    density=True,
                )

            # 植生クラス（緑色）
            ax.hist(
                rgb_veg[:, ch],
                bins=30,
                alpha=0.5,
                label=f"{veg_class} (植生)",
                color="#4CAF50",
                density=True,
            )

            # 平均値の縦線
            if len(rgb_water_correct) > 0:
                ax.axvline(
                    np.mean(rgb_water_correct[:, ch]),
                    color="#1565C0",
                    linestyle="--",
                    linewidth=2,
                )
            if len(rgb_water_wrong) > 0:
                ax.axvline(
                    np.mean(rgb_water_wrong[:, ch]),
                    color="#C62828",
                    linestyle="-",
                    linewidth=3,
                    label="誤分類の平均",
                )
            ax.axvline(
                np.mean(rgb_veg[:, ch]), color="#2E7D32", linestyle="--", linewidth=2
            )

            ax.set_xlabel(f"{ch_name} Value (0-255)", fontsize=11)
            ax.set_ylabel("Density", fontsize=11)
            ax.set_title(
                f"{ch_name} Channel\n({water_class} vs {veg_class})",
                fontsize=12,
                fontweight="bold",
            )
            ax.legend(fontsize=8, loc="upper right")
            ax.grid(True, alpha=0.3)

            # 統計分析
            if len(rgb_water_wrong) > 0 and len(rgb_water_correct) > 0:
                mean_correct = np.mean(rgb_water_correct[:, ch])
                mean_wrong = np.mean(rgb_water_wrong[:, ch])
                mean_veg = np.mean(rgb_veg[:, ch])

                print(f"\n  【{ch_name}チャンネル】")
                print(f"    {water_class} 正解: {mean_correct:.1f}")
                print(f"    {water_class} 誤分類: {mean_wrong:.1f}")
                print(f"    {veg_class}: {mean_veg:.1f}")

                # 誤分類サンプルが植生に近いか？
                dist_to_correct = abs(mean_wrong - mean_correct)
                dist_to_veg = abs(mean_wrong - mean_veg)

                if ch == 1:  # Green チャンネルが特に重要
                    if dist_to_veg < dist_to_correct:
                        print(
                            f"    → 🔴 誤分類サンプルは{veg_class}に近い色（緑が強い）"
                        )
                    else:
                        print(f"    → 🟢 誤分類サンプルは元の{water_class}に近い色")

        # --- 2. Green vs Blue 2Dプロット（水域と植生の分離に重要）---
        ax = axes[row, 3]

        # 正しく分類された水域
        if len(rgb_water_correct) > 0:
            ax.scatter(
                rgb_water_correct[:, 1],
                rgb_water_correct[:, 2],
                alpha=0.3,
                label=f"{water_class} (正解)",
                c="#2196F3",
                s=15,
            )

        # 植生と誤分類された水域（大きく強調）
        if len(rgb_water_wrong) > 0:
            ax.scatter(
                rgb_water_wrong[:, 1],
                rgb_water_wrong[:, 2],
                alpha=1.0,
                label=f"{water_class}→{veg_class} (誤分類)",
                c="#F44336",
                s=80,
                marker="X",
                edgecolors="black",
                linewidth=1,
            )

        # 植生クラス
        ax.scatter(
            rgb_veg[:, 1],
            rgb_veg[:, 2],
            alpha=0.3,
            label=f"{veg_class}",
            c="#4CAF50",
            s=15,
        )

        # 重心
        if len(rgb_water_correct) > 0:
            center_water = np.mean(rgb_water_correct, axis=0)
            ax.scatter(
                center_water[1],
                center_water[2],
                marker="★",
                s=300,
                c="#1565C0",
                edgecolors="white",
                linewidth=2,
                zorder=10,
            )

        if len(rgb_water_wrong) > 0:
            center_wrong = np.mean(rgb_water_wrong, axis=0)
            ax.scatter(
                center_wrong[1],
                center_wrong[2],
                marker="★",
                s=300,
                c="#C62828",
                edgecolors="white",
                linewidth=2,
                zorder=10,
            )

        center_veg = np.mean(rgb_veg, axis=0)
        ax.scatter(
            center_veg[1],
            center_veg[2],
            marker="★",
            s=300,
            c="#2E7D32",
            edgecolors="white",
            linewidth=2,
            zorder=10,
        )

        ax.set_xlabel("Green Mean", fontsize=11)
        ax.set_ylabel("Blue Mean", fontsize=11)
        ax.set_title(
            f"Green vs Blue\n誤分類サンプルは緑方向にシフト？",
            fontsize=12,
            fontweight="bold",
        )
        ax.legend(fontsize=9, loc="upper left")
        ax.grid(True, alpha=0.3)

        # 定量分析
        if len(rgb_water_wrong) > 0 and len(rgb_water_correct) > 0:
            center_water = np.mean(rgb_water_correct, axis=0)
            center_wrong = np.mean(rgb_water_wrong, axis=0)
            center_veg = np.mean(rgb_veg, axis=0)

            dist_wrong_to_water = np.linalg.norm(center_wrong - center_water)
            dist_wrong_to_veg = np.linalg.norm(center_wrong - center_veg)

            print(f"\n  【RGB空間での位置関係】")
            print(f"    誤分類サンプル → {water_class}重心: {dist_wrong_to_water:.2f}")
            print(f"    誤分類サンプル → {veg_class}重心: {dist_wrong_to_veg:.2f}")

            # Green成分の比較
            green_shift = center_wrong[1] - center_water[1]
            print(f"    Green成分のシフト: {green_shift:+.2f}")

            if dist_wrong_to_veg < dist_wrong_to_water:
                print(f"\n    ✅ 結論: 誤分類サンプルはRGB空間で{veg_class}に近い")
                print(
                    f"       → 緑色成分が強い{water_class}画像が{veg_class}と誤認された"
                )
            else:
                print(f"\n    ❌ 誤分類サンプルは{water_class}の領域内にある")
                print(f"       → 色以外の要因（テクスチャ等）が誤分類の原因の可能性")

    plt.suptitle(
        "水域の緑色成分と植生への誤分類の関係\n（赤X = 誤分類サンプル）",
        fontsize=16,
        fontweight="bold",
        y=1.02,
    )
    plt.tight_layout()
    plt.savefig("water_vegetation_misclassification.png", dpi=300, bbox_inches="tight")
    print(f"\n図を保存しました: water_vegetation_misclassification.png")
    plt.show()

    # 総括
    print("\n" + "=" * 70)
    print("【総括】")
    print("=" * 70)
    print("""
■ 仮説の検証結果:
  水域（SeaLake, River）に緑色成分（藻類、水草等）が存在する場合、
  緑系植生（Pasture, AnnualCrop）と誤分類されやすい。

■ 誤分類のメカニズム:
  1. 通常の水域: 青色が強く、Green値が低い → 正しく水域と判定
  2. 緑がかった水域: 藻類等でGreen値が上昇 → 植生に似た色になる
  3. モデルの判断: 「緑が強い = 植物」と学習しているため誤分類

■ 対策案:
  • テクスチャ特徴の活用（水面の反射パターン vs 植生のテクスチャ）
  • 空間的特徴の考慮（湖の形状 vs 植物の不均一な分布）
  • NIR（近赤外線）バンドの追加（水と植物は反射特性が異なる）
""")


# 実行
compare_water_vs_vegetation(x_test, y_test, pred_labels, class_names)